# 1.- Definición General


## intención del colab

La diferencia con el colab de  dataset básico es que voy a dejar


*   completas las series con 0 ventas en el medio
*   dejar el dataset por cliente producto partido por clusters


#### Incluye
*   Datos básicos del problema
*   Datos de insights del problema
*   Datos de Argentina (sin adivinar futuro)



#### NO incluye
*   Deltas, lags
*   relaciones estadísticas: medias, desvíos, min, max, ponderados y periódicos
*   Correlaciones entre series
*   Trabajos con NA / ceros





# 2.- Datos básicos del problema df_base_30

### Ambito general

In [92]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [93]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend

#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

### Funciones útiles, gracias Diana Bermudez


In [94]:
def eliminar_columnas_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina columnas de un DataFrame que contienen exclusivamente el valor cero.

    Args:
        df (pd.DataFrame): El DataFrame de entrada con múltiples columnas.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las columnas que no eran solo ceros.
                      Si no hay columnas restantes, devuelve un DataFrame vacío.
    """
    # Identificar las columnas que contienen solo ceros
    # Para cada columna, verificamos si la suma de sus valores absolutos es cero.
    # Esto maneja tanto ceros enteros como ceros flotantes.
    columnas_a_mantener = [col for col in df.columns if df[col].abs().sum() != 0]

    # Crear un nuevo DataFrame solo con las columnas a mantener
    df_resultante = df[columnas_a_mantener].copy()

    return df_resultante

In [95]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

### Carga de archivos datos

In [96]:
#cargar csv
# ventas al canal, va a ser el archivo padre
sellin = pd.read_csv(DATOS_DIR +'sell-in.txt', sep='\t')
# productos a validar, por ahora no se me ocurre como usarlos, pero los cargo por las dudas. Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# cargo la tabla de características
prod_cat = pd.read_csv(DATOS_DIR +'tb_productos.txt', sep='\t')
# cargo los stocks de los productos
stocks = pd.read_csv(DATOS_DIR +'tb_stocks.txt', sep='\t')
# me traigo el clasificador de clusters
df_llave= pd.read_csv(DATOS_DIR +'df_llave_cluster_93.csv')

### proceso básico de sell-in => df_base_1

In [97]:
print(sellin.shape)
sellin.head()

(2945818, 7)


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.053,0.053
1,201701,10032,20524,0,1,0.136,0.136
2,201701,10217,20524,0,1,0.030,0.030
3,201701,10125,20524,0,1,0.023,0.023
4,201701,10012,20524,0,11,1.545,1.545


### resulta que ahora yo voy a ir por producto solamente

In [98]:
# cust request no me sirve.
#drop de la columna cust_request_qty
sellin = sellin.drop(columns=['cust_request_qty'])
sellin.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_tn,tn
0,201701,10234,20524,0,0.053,0.053
1,201701,10032,20524,0,0.136,0.136
2,201701,10217,20524,0,0.030,0.030
3,201701,10125,20524,0,0.023,0.023
4,201701,10012,20524,0,1.545,1.545


In [99]:
# ahora voy a hacer df_base1 = sellin
df_base_1 = sellin.copy()

In [100]:
# dropeo la columna customer
df_base_1 = df_base_1.drop(columns=['customer_id'])
df_base_1.head()

,periodo,product_id,plan_precios_cuidados,cust_request_tn,tn
0,201701,20524,0,0.053,0.053
1,201701,20524,0,0.136,0.136
2,201701,20524,0,0.030,0.030
3,201701,20524,0,0.023,0.023
4,201701,20524,0,1.545,1.545


In [101]:
df_base_1.shape

(2945818, 5)

In [102]:
# quiero un solo registro por valor único de período. tn se agrega como suma, cust_request_tn se agrega como suma, plan_precios_cuidados se dropea
df_base_1 = df_base_1.groupby(['periodo', 'product_id']).agg({
    'tn': 'sum',
    'cust_request_tn': 'sum',
    'plan_precios_cuidados': 'first'
}).reset_index()
df_base_1.head()

,periodo,product_id,tn,cust_request_tn,plan_precios_cuidados
0,201701,20001,934.772,937.727,0
1,201701,20002,550.157,555.187,0
2,201701,20003,1063.458,1067.815,0
3,201701,20004,555.916,569.374,0
4,201701,20005,494.270,494.601,0


In [103]:
df_base_1.shape

(31243, 5)

In [104]:
# lo primero que hago es eliminar cust_request_qty porque debe er la relación entre peso y cust request tn
# y si dejo cust_request_tn va a ser demasiando igual a tn, por lo que voy a dejar solamente dif_tn y dif_por como las diferencias absolutas y porcentuales sobre lo pedido
df_base_1['dif_tn'] = df_base_1['tn'] - df_base_1['cust_request_tn']
df_base_1['dif_por'] = df_base_1['dif_tn'] / df_base_1['cust_request_tn']
df_base_1 = df_base_1.drop(columns=['cust_request_tn'])
df_base_1.head()

,periodo,product_id,tn,plan_precios_cuidados,dif_tn,dif_por
0,201701,20001,934.772,0,-2.955,-3.151e-03
1,201701,20002,550.157,0,-5.029,-9.059e-03
2,201701,20003,1063.458,0,-4.357,-4.080e-03
3,201701,20004,555.916,0,-13.458,-2.364e-02
4,201701,20005,494.270,0,-0.331,-6.687e-04


In [105]:
# como no sé bien dónde va a caer precios cuidados cuando complete con ceros y como a nadie le dió importante
# dropeo la columna de precios cuidados
df_base_1 = df_base_1.drop(columns=['plan_precios_cuidados'])
df_base_1.head()

,periodo,product_id,tn,dif_tn,dif_por
0,201701,20001,934.772,-2.955,-3.151e-03
1,201701,20002,550.157,-5.029,-9.059e-03
2,201701,20003,1063.458,-4.357,-4.080e-03
3,201701,20004,555.916,-13.458,-2.364e-02
4,201701,20005,494.270,-0.331,-6.687e-04


In [106]:
## de la columna periodo tengo que generar mes, año, trimestre, date(formato daytime), secuencia que es el mes consecutivo
## genero también una columna date_mes en formato período que me facilite la vida más adelante.
df_base_1['anio']=df_base_1['periodo'] //100
df_base_1['mes']=df_base_1['periodo'] % 100
df_base_1['secuencia']= (df_base_1['anio']-2017)*12+df_base_1['mes']
df_base_1['trimestre']=df_base_1['mes']//3
df_base_1['date'] = pd.to_datetime(df_base_1['periodo'], format='%Y%m')
df_base_1['date_mes'] = df_base_1['date'].dt.to_period('M')
df_base_1.head()


,periodo,product_id,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes
0,201701,20001,934.772,-2.955,-3.151e-03,2017,1,1,0,2017-01-01,2017-01
1,201701,20002,550.157,-5.029,-9.059e-03,2017,1,1,0,2017-01-01,2017-01
2,201701,20003,1063.458,-4.357,-4.080e-03,2017,1,1,0,2017-01-01,2017-01
3,201701,20004,555.916,-13.458,-2.364e-02,2017,1,1,0,2017-01-01,2017-01
4,201701,20005,494.270,-0.331,-6.687e-04,2017,1,1,0,2017-01-01,2017-01


In [107]:
# para simplificar voy a eliminar las columnas anio mes secuencia trimestre date periodo
df_base_1 = df_base_1.drop(columns=['anio', 'mes', 'secuencia', 'trimestre', 'date', 'periodo'])
df_base_1.head()

,product_id,tn,dif_tn,dif_por,date_mes
0,20001,934.772,-2.955,-3.151e-03,2017-01
1,20002,550.157,-5.029,-9.059e-03,2017-01
2,20003,1063.458,-4.357,-4.080e-03,2017-01
3,20004,555.916,-13.458,-2.364e-02,2017-01
4,20005,494.270,-0.331,-6.687e-04,2017-01


In [108]:
# veo los types
df_base_1.dtypes

,0
product_id,int64
tn,float64
dif_tn,float64
dif_por,float64
date_mes,period[M]


In [109]:
df_base_1.head()

,product_id,tn,dif_tn,dif_por,date_mes
0,20001,934.772,-2.955,-3.151e-03,2017-01
1,20002,550.157,-5.029,-9.059e-03,2017-01
2,20003,1063.458,-4.357,-4.080e-03,2017-01
3,20004,555.916,-13.458,-2.364e-02,2017-01
4,20005,494.270,-0.331,-6.687e-04,2017-01


### ahora quiero completar los ceros de productos  y  customer

Tengo una matriz larga de ventas con muchas columnas. Entre ellas tengo customer_id que identifica el cliente y product_id que identifica el producto. Además tengo una columna date formato YYYY-MM que refleja la fecha de la venta, una columna tn que es la venta y una serie de columnas cuyo valor está relacionado al producto y una serie de columnas cuyo valor está relacionado al cliente. Son las ventas realmente efectuadas. Para el análisis siguiente tengo que ser capaz de distinguir si una venta no se produjo porque el cliente o el producto no existían o porque el cliente decidió no comprar. Por ello tengo que realizar las siguientes operaciones. Voy a considerar que un cliente está activo a partir de su primera compra. Y que un producto está activo a partir de su primera venta. Necesito entonces llenar el df existente completando las ventas de 0 tn de todo producto activo para todo cliente activo que no lo haya comprado en cada período. parafrasear para verificar entendimiento y generar código python comentado

In [110]:
df_base_1.dtypes

,0
product_id,int64
tn,float64
dif_tn,float64
dif_por,float64
date_mes,period[M]


In [111]:
# 2. Determina la primera y última fecha de venta de cada producto
first_product_date = df_base_1.groupby('product_id')['date_mes'].min().rename('first_product_date')
last_product_date = df_base_1.groupby('product_id')['date_mes'].max().rename('last_product_date')

In [112]:
# 3. Consigue todos los meses disponibles
all_periods = df_base_1['date_mes'].sort_values().unique()

In [113]:
# 4. Saca todas las combinaciones posibles (producto, cliente, mes)

productos = df_base_1['product_id'].unique()
grid = pd.MultiIndex.from_product(
    [ productos, all_periods],
    names=['product_id', 'date_mes']
)
full_df = pd.DataFrame(index=grid).reset_index()

In [114]:
# 5. Añade las fechas de activación y último registro
full_df = (full_df
    .merge(first_product_date, on='product_id')
    .merge(last_product_date,  on='product_id')
)

In [115]:
# 6. Filtra SOLO donde ambos están activos en ese mes
mask = (
    (full_df['date_mes'] >= full_df['first_product_date'])
)
full_df = full_df[mask].copy()

In [116]:
# 7. Une tu base original para saber cuándo hubo venta
full_df = full_df.merge(
    df_base_1[[ 'product_id', 'date_mes', 'tn', 'dif_tn', 'dif_por']],
    on=[ 'product_id', 'date_mes'],
    how='left'
)

In [117]:
# 8. Donde no hay venta, rellena con cero
for col in ['tn', 'dif_tn', 'dif_por']:
    full_df[col] = full_df[col].fillna(0)

In [118]:
full_df.shape

(35888, 7)

In [ ]:
df_base_1.shape

(31243, 5)

In [119]:
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031


In [120]:
# agrego el tiempo de vida del p´roducto
primer_mes = full_df['date_mes'].min()

def calcular_vida_prod(row):
    # Si el producto existe desde el primer mes, aplicar mínimo de 4
    if row['first_product_date'] == primer_mes:
        vida = (row['date_mes'] - row['first_product_date']).n + 4
    else:
        vida = (row['date_mes'] - row['first_product_date']).n + 1  # sumar 1 para contar el mes inicial
    return max(vida, 1)  # Solo por seguridad, nunca sea menor que 1

full_df['vida_prod'] = full_df.apply(calcular_vida_prod, axis=1)
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8


In [121]:
full_df.dtypes

,0
product_id,int64
date_mes,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64
vida_prod,int64


In [122]:
full_df.tail()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod
35883,20728,2019-11,2019-11,2019-12,16.022,0.0,0.0,1
35884,20728,2019-12,2019-11,2019-12,14.316,0.0,0.0,2
35885,20770,2019-12,2019-12,2019-12,3.185,0.0,0.0,1
35886,20792,2019-12,2019-12,2019-12,3.809,0.0,0.0,1
35887,20854,2019-12,2019-12,2019-12,3.964,0.0,0.0,1


In [123]:
# Asume full_df ya está ordenado por customer_id, product_id, date_mes
full_df = full_df.sort_values([ 'product_id', 'date_mes']).reset_index(drop=True)

# Creamos columnas vacías
full_df['meses_on'] = 0
full_df['meses_off'] = 0

# Calculamos el primer período global
primer_mes = full_df['date_mes'].min()

# Función que aplica la lógica mencionada sobre cada grupo cliente-producto
def acumular_on_off(grp):
    meses_on = []
    meses_off = []
    on_count = 0
    off_count = 0
    for i, row in grp.iterrows():
        # Primer período
        if row['date_mes'] == primer_mes:
            if row['tn'] != 0:
                on_count = 4
                off_count = 0
            else:
                on_count = 0
                off_count = 4
        else:
            if row['tn'] != 0:
                on_count = on_count + 1 if on_count > 0 else 1
                off_count = 0
            else:
                on_count = 0
                off_count = off_count + 1 if off_count > 0 else 1
        meses_on.append(on_count)
        meses_off.append(off_count)
    grp['meses_on'] = meses_on
    grp['meses_off'] = meses_off
    return grp

# Aplicamos la función a cada grupo de cliente-producto
full_df = full_df.groupby(['product_id'], group_keys=False).apply(acumular_on_off)

# ¡Listo! Ahora full_df tiene las columnas meses_on y meses_off con la lógica indicada.

/tmp/ipython-input-123-2099794536.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df = full_df.groupby(['product_id'], group_keys=False).apply(acumular_on_off)


In [124]:
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0


In [125]:
# 1. Creamos un set para búsqueda eficiente
productos_indeterminados = set(productos_validar['product_id'].values)

# 2. Función para asignar vida_restante por fila
def calcular_vida_restante(row):
    if row['product_id'] in productos_indeterminados:
        return 60
    else:
        # (asumiendo que las columnas son period[M])
        return max(0, (row['last_product_date'] - row['date_mes']).n)
        # max(0, ...) por si date_mes supera last_product_date nunca sea negativo

# 3. Aplicamos la función fila a fila
full_df['vida_restante'] = full_df.apply(calcular_vida_restante, axis=1)

# Listo, full_df ahora tiene la columna vida_restante según tus reglas.

In [126]:
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60


In [127]:
# Lista de columnas calculadas
cols = [
      'first_product_date', 'last_product_date',
    'vida_prod',  'meses_on', 'meses_off', 'vida_restante'
]

# Mostramos la cantidad de valores únicos por columna calculada
for col in cols:
    print(f'{col}: {full_df[col].nunique()} valores únicos')

first_product_date: 33 valores únicos
last_product_date: 35 valores únicos
vida_prod: 39 valores únicos
meses_on: 40 valores únicos
meses_off: 36 valores únicos
vida_restante: 37 valores únicos


In [128]:
full_df.describe()

,product_id,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante
count,35888.000,35888.000,35888.000,35888.000,35888.000,35888.000,35888.000,35888.000
mean,20583.567,36.920,-0.783,-0.006,19.509,15.768,1.386,40.757
std,369.165,103.422,5.023,0.025,10.713,11.610,4.608,25.490
min,20001.000,0.000,-190.705,-0.960,1.000,0.000,0.000,0.000
25%,20267.000,0.989,0.000,0.000,10.000,6.000,0.000,12.000
50%,20552.000,6.704,0.000,0.000,19.000,14.000,0.000,60.000
75%,20878.000,24.271,0.000,0.000,28.000,25.000,0.000,60.000
max,21299.000,2295.198,14.347,0.366,39.000,39.000,35.000,60.000


In [129]:
# genero una columna anio como integrer a partir de date_mes
full_df['anio'] = full_df['date_mes'].dt.year
full_df['mes'] = full_df['date_mes'].dt.month
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5


In [130]:
# genero la columna secuencia
full_df['secuencia']= (full_df['anio']-2017)*12+full_df['mes']
full_df['trimestre']=full_df['date_mes'].dt.quarter
full_df.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1,1,1
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2,2,1
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3,3,1
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4,4,2
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5,5,2


In [131]:
full_df.shape

(35888, 15)

### datos básico categoría de productos y luego unión para df_base_2


In [132]:
# busco y elimino registros duplicados
# Supongamos que tu columna se llama 'columna'
duplicados = prod_cat['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat['product_id'][prod_cat['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: True
Valores duplicados:
[20010 20022 20021 20020 20278 20192 20037 20105 20100 20623 20230]


In [133]:
#veo los duplicados a ver si amerita algún tratamiento o son simple error
duplicadas_filas = prod_cat[prod_cat['product_id'].duplicated(keep=False)]
print(duplicadas_filas)

      cat1            cat2            cat3     brand  sku_size  product_id
36      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
37      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
38      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
39      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
41      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
42      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
43      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
44      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
541  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
542  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
550  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
551  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
552  FOODS  SOPAS Y CALDO

In [134]:
prod_cat.isnull().sum()

,0
cat1,0
cat2,0
cat3,0
brand,0
sku_size,0
product_id,0


In [135]:
#elimino los duplicados
# el peor caso es un shampoo por un acondicionador... no cambia mucho
prod_cat_unic = prod_cat.drop_duplicates(subset=['product_id'])
duplicados = prod_cat_unic['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat_unic['product_id'][prod_cat_unic['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: False
Valores duplicados:
[]


In [136]:
#voy metiendo todo en una sola matriz,
# que me llevo
columnas_a_agregar = ['cat1', 'cat2', 'cat3','brand','sku_size', 'product_id']
# resultado
df_base_2 = full_df.merge(prod_cat_unic[columnas_a_agregar], on='product_id', how='left')

df_base_2.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0


In [137]:
df_base_2.shape

(35888, 20)

In [138]:
df_base_2.isnull().sum()

,0
product_id,0
date_mes,0
first_product_date,0
last_product_date,0
tn,0
dif_tn,0
dif_por,0
vida_prod,0
meses_on,0
meses_off,0


In [139]:
# tengo el problema que hay 1432 instancias donde no puedo asociar producto con características
# posiblement los productos que fueron mal colocados
# los voy a reemplazar por UNKN por desconocido
columnas_raras =['cat1', 'cat2', 'cat3','brand']
df_base_2[columnas_raras] = df_base_2[columnas_raras].fillna('UNKN')
df_base_2["sku_size"] = df_base_2["sku_size"].fillna(0)
df_base_2.isnull().sum()
#

,0
product_id,0
date_mes,0
first_product_date,0
last_product_date,0
tn,0
dif_tn,0
dif_por,0
vida_prod,0
meses_on,0
meses_off,0


In [140]:
df_base_2.tail()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size
35883,21299,2019-08,2017-08,2017-08,0.0,0.0,0.0,25,0,24,0,2019,8,32,3,UNKN,UNKN,UNKN,UNKN,0.0
35884,21299,2019-09,2017-08,2017-08,0.0,0.0,0.0,26,0,25,0,2019,9,33,3,UNKN,UNKN,UNKN,UNKN,0.0
35885,21299,2019-10,2017-08,2017-08,0.0,0.0,0.0,27,0,26,0,2019,10,34,4,UNKN,UNKN,UNKN,UNKN,0.0
35886,21299,2019-11,2017-08,2017-08,0.0,0.0,0.0,28,0,27,0,2019,11,35,4,UNKN,UNKN,UNKN,UNKN,0.0
35887,21299,2019-12,2017-08,2017-08,0.0,0.0,0.0,29,0,28,0,2019,12,36,4,UNKN,UNKN,UNKN,UNKN,0.0


# 3.- Datos de insights del problema

### en clase se habló de ver hermanos mayores y menores, mismo producto distinto size, no queda muy claro como hacerlo a partir de la codificación, por ahora no lo hago


### la estructura de categorías, el árbol no me resulta claro, por lo que voy a crear nuevas categorías que lo reflejen df_base_3


In [141]:
# cuando yo voy partiendo en categorías la idea es ir quedandome con subconjuntos lo más parecidos posibles
# y nada me garantiza la unicidad o correspondencia de las catagorías que aquí aparecen más abajo
# por ejemplo puedo tener un jabón de 500 ml  para bebé y uno de lavar la ropa de 500ml, no me hace sentido agruparlos solamente por su size
# entonces voy a generar Categorías a partir de la 2 que sean una concatenación de los valores de más arriba en la jerarquia
df_base_3=df_base_2.copy()
df_base_3['cat1_cat2'] = df_base_3['cat1'].fillna('') + '_' + df_base_3['cat2'].fillna('')
df_base_3['cat1_cat2_cat3'] = df_base_3['cat1_cat2'].fillna('') + '_' + df_base_3['cat3'].fillna('')
df_base_3['cat1_cat2_cat3_brand'] = df_base_3['cat1_cat2_cat3'].fillna('') + '_' + df_base_3['brand'].fillna('')
df_base_3['cat1_cat2_cat3_brand_size'] = df_base_3['cat1_cat2_cat3_brand'].fillna('') + '_' + df_base_3['sku_size'].astype(str)
df_base_3.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0


In [142]:
# agrego stock a fin de mes por producto, no me interesa en esta instancia generar los deltas , medias etc)
print(stocks.tail())

       periodo  product_id  stock_final
13686   201912       20453        1.437
13687   201912       21026        7.268
13688   201912       21054        0.508
13689   201912       20981        2.185
13690   201912       20853        1.824


In [143]:
# tengo que reconstruir la columna periodo para poder hacer el merge
# defino periodo para df_base_3 como un integrer que es el anio multiplicado por 100 más el mes
df_base_3['periodo'] = df_base_3['anio'] * 100 + df_base_3['mes']
df_base_3.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705


In [144]:
# 2. Añadir el stock. La clave de unión es el mes Y el producto.
df_base_3 = pd.merge(df_base_3, stocks, on=['periodo', 'product_id'], how='left')
df_base_3.tail()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final
35883,21299,2019-08,2017-08,2017-08,0.0,0.0,0.0,25,0,24,0,2019,8,32,3,UNKN,UNKN,UNKN,UNKN,0.0,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_0.0,201908,NaN
35884,21299,2019-09,2017-08,2017-08,0.0,0.0,0.0,26,0,25,0,2019,9,33,3,UNKN,UNKN,UNKN,UNKN,0.0,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_0.0,201909,NaN
35885,21299,2019-10,2017-08,2017-08,0.0,0.0,0.0,27,0,26,0,2019,10,34,4,UNKN,UNKN,UNKN,UNKN,0.0,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_0.0,201910,NaN
35886,21299,2019-11,2017-08,2017-08,0.0,0.0,0.0,28,0,27,0,2019,11,35,4,UNKN,UNKN,UNKN,UNKN,0.0,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_0.0,201911,NaN
35887,21299,2019-12,2017-08,2017-08,0.0,0.0,0.0,29,0,28,0,2019,12,36,4,UNKN,UNKN,UNKN,UNKN,0.0,UNKN_UNKN,UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN,UNKN_UNKN_UNKN_UNKN_0.0,201912,NaN


In [145]:
df_base_3.to_parquet(DATOS_DIR +'df_base_3_prod.parquet', index=False)

In [146]:
df_base_3.dtypes


,0
product_id,int64
date_mes,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64
vida_prod,int64
meses_on,int64
meses_off,int64


## Ahora voy a agregar los datos de argentina

In [147]:
df_base_20 =pd.read_csv(DATOS_DIR +'df_base_20.csv')
df_base_20.head()

,periodo,date,date_mes,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,201701,2017-01-01 00:00:00,2017-01,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,201702,2017-02-01 00:00:00,2017-02,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,201703,2017-03-01 00:00:00,2017-03,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,201704,2017-04-01 00:00:00,2017-04,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,201705,2017-05-01 00:00:00,2017-05,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [148]:
# elimino las columnas date y date_mes de df_base_20
df_base_20 = df_base_20.drop(columns=['date', 'date_mes'])
df_base_20.head()

,periodo,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,201701,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,201702,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,201703,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,201704,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,201705,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [149]:
df_base_20.dtypes

,0
periodo,int64
dias_laborables_arg,int64
cisne_negro,int64
VtaGral,float64
VtaGralCte,float64
Bebidas,float64
Almacén,float64
Panadería,float64
Lácteos,float64
Carnes,float64


In [150]:
df_base_3 = df_base_3.merge(df_base_20, on='periodo', how='left')

In [151]:
df_base_3.head()

,product_id,date_mes,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,20001,2017-01,2017-01,2019-12,934.772,-2.955,-0.003,4,4,0,60,2017,1,1,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,20001,2017-02,2017-01,2019-12,798.016,-35.706,-0.043,5,5,0,60,2017,2,2,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201702,NaN,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,20001,2017-03,2017-01,2019-12,1303.358,-27.389,-0.021,6,6,0,60,2017,3,3,1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201703,NaN,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,20001,2017-04,2017-01,2019-12,1069.961,-62.983,-0.056,7,7,0,60,2017,4,4,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201704,NaN,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,20001,2017-05,2017-01,2019-12,1502.201,-48.488,-0.031,8,8,0,60,2017,5,5,2,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC_ROPA LAVADO,HC_ROPA LAVADO_Liquido,HC_ROPA LAVADO_Liquido_ARIEL,HC_ROPA LAVADO_Liquido_ARIEL_3000.0,201705,NaN,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


In [152]:
df_base_3.shape

(35888, 44)

In [154]:
df_base_3.to_parquet(DATOS_DIR +'df_prod_base.parquet', index=False)
